In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

from database_operations import DatabaseOperations
from overtake_model import OvertakingModel
from race_data import RaceDataSetup
from race_dataframe import RaceDataframe
from race_sim import RaceSimulator
from evaluation import RaceSimEvaluation, EvaluateMany
from optimisation import Optimisation

import time
from joblib import Parallel, delayed

import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None) 

## Mexico GP

In [2]:
# Initialise all classes
database_ops_mexico = DatabaseOperations(2024, "Mexico City")
race_df_mexico = RaceDataframe(database_ops_mexico)
race_data_mexico = RaceDataSetup(database_ops_mexico, race_df_mexico)
overtake_model_mexico = OvertakingModel(race_df_mexico.race_df)
simulation_mexico = RaceSimulator(race_data_mexico, overtake_model_mexico)

driver = 44
optimise_mexico_44 = Optimisation(race_data_mexico, overtake_model_mexico, driver)
driver = 63
optimise_mexico_63 = Optimisation(race_data_mexico, overtake_model_mexico, driver)

## Singapore GP

In [3]:
# Initialise all classes
database_ops_singapore = DatabaseOperations(2024, "Marina Bay")
race_df_singapore = RaceDataframe(database_ops_singapore)
race_data_singapore = RaceDataSetup(database_ops_singapore, race_df_singapore)
overtake_model_singapore = OvertakingModel(race_df_singapore.race_df)
simulation_singapore = RaceSimulator(race_data_singapore, overtake_model_singapore)

driver = 44
optimise_singapore_44 = Optimisation(race_data_singapore, overtake_model_singapore, driver)

driver = 63
optimise_singapore_63 = Optimisation(race_data_singapore, overtake_model_singapore, driver)

## Spanish GP

In [4]:
# Initialise all classes
database_ops_spain = DatabaseOperations(2024, "Barcelona")
race_df_spain = RaceDataframe(database_ops_spain)
race_data_spain = RaceDataSetup(database_ops_spain, race_df_spain)
overtake_model_spain = OvertakingModel(race_df_spain.race_df)
simulation_spain = RaceSimulator(race_data_spain, overtake_model_spain)

driver = 44
optimise_spain_44 = Optimisation(race_data_spain, overtake_model_spain, driver)
driver = 63
optimise_spain_63 = Optimisation(race_data_spain, overtake_model_spain, driver)

## Run Opt

In [5]:
def run_optimisation(optimise, max_iterations, population_size, generations):
	start_time_bayesian = time.perf_counter()
	strategies_bayesian = optimise.bayesian_optimisation(max_iterations=max_iterations)
	runtime_bayesian = time.perf_counter() - start_time_bayesian
	
	start_time_genetic = time.perf_counter()
	strategies_genetic = optimise.genetic_algorithm_optimisation(population_size=population_size, generations=generations)
	runtime_genetic = time.perf_counter() - start_time_genetic
	
	actual_strategy = optimise.get_actual_strategy()

	return strategies_bayesian, runtime_bayesian, strategies_genetic, runtime_genetic, actual_strategy

In [10]:
def print_results(race_name, strategies_bayesian, runtime_bayesian, strategies_genetic, runtime_genetic, actual_strategy_data):
    print(f"=== {race_name} Results ===")
    print(f"Bayesian Optimization Runtime: {runtime_bayesian:.2f} seconds")
    print("Top 10 Bayesian Optimization Strategies:")
    for i, strategy in enumerate(strategies_bayesian[:10], start=1):
        print(f"  {i}. Strategy: {strategy['strategy']}, Final Position: {strategy['position']}")

    print(f"\nGenetic Algorithm Runtime: {runtime_genetic:.2f} seconds")
    print("Top 10 Genetic Algorithm Strategies:")
    for i, strategy in enumerate(strategies_genetic[:10], start=1):
        print(f"  {i}. Strategy: {strategy['strategy']}, Final Position: {strategy['final_position']}")

    print("\nActual Strategy:")
    print(f"  Strategy: {actual_strategy_data['actual_strat']}")
    print(f"  Actual Finishing Position: {actual_strategy_data['actual_finishing_pos']}")
    print(f"  Simualted Finishing Position: {actual_strategy_data['simualted_finishing_pos']}")

    print("=" * 40)

In [7]:
from concurrent.futures import ThreadPoolExecutor

optimisation_params = (200, 50, 6)
# optimisation_params = (10, 5, 1)

# Run optimizations in parallel using threads
with ThreadPoolExecutor(max_workers=6) as executor:
    mexico_44 = executor.submit(run_optimisation, optimise_mexico_44, *optimisation_params)
    singapore_44 = executor.submit(run_optimisation, optimise_singapore_44, *optimisation_params)
    spain_44 = executor.submit(run_optimisation, optimise_spain_44, *optimisation_params)

    mexico_63 = executor.submit(run_optimisation, optimise_mexico_63, *optimisation_params)
    singapore_63 = executor.submit(run_optimisation, optimise_singapore_63, *optimisation_params)
    spain_63 = executor.submit(run_optimisation, optimise_spain_63, *optimisation_params)

    # Retrieve results
    mexico_result_44 = mexico_44.result()
    singapore_result_44 = singapore_44.result()
    spain_result_44 = spain_44.result()
    
    # Retrieve results
    mexico_result_63 = mexico_63.result()
    singapore_result_63 = singapore_63.result()
    spain_result_63 = spain_63.result()

mexico_bayesian_44, mexico_bayesian_runtime_44, mexico_genetic_44, mexico_genetic_runtime_44, mexico_actual_44 = mexico_result_44
singapore_bayesian_44, singapore_bayesian_runtime_44, singapore_genetic_44, singapore_genetic_runtime_44, singapore_actual_44 = singapore_result_44
spain_bayesian_44, spain_bayesian_runtime_44, spain_genetic_44, spain_genetic_runtime_44, spain_actual_44 = spain_result_44

mexico_bayesian_63, mexico_bayesian_runtime_63, mexico_genetic_63, mexico_genetic_runtime_63, mexico_actual_63 = mexico_result_63
singapore_bayesian_63, singapore_bayesian_runtime_63, singapore_genetic_63, singapore_genetic_runtime_63, singapore_actual_63 = singapore_result_63
spain_bayesian_63, spain_bayesian_runtime_63, spain_genetic_63, spain_genetic_runtime_63, spain_actual_63 = spain_result_63

|   iter    |  target   | num_pi... | pit1_lap  | pit1_tyre | pit2_lap  | pit2_tyre | pit3_lap  | pit3_tyre | start_... |
-------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | num_pi... | pit1_lap  | pit1_tyre | pit2_lap  | pit2_tyre | pit3_lap  | pit3_tyre | start_... |
-------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | num_pi... | pit1_lap  | pit1_tyre | pit2_lap  | pit2_tyre | pit3_lap  | pit3_tyre | start_... |
-------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | num_pi... | pit1_lap  | pit1_tyre | pit2_lap  | pit2_tyre | pit3_lap  | pit3_tyre | start_... |
-------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   

In [11]:
print_results("Mexico GP", mexico_bayesian_44, mexico_bayesian_runtime_44, mexico_genetic_44, mexico_genetic_runtime_44, mexico_actual_44)
print_results("Singapore GP", singapore_bayesian_44, singapore_bayesian_runtime_44, singapore_genetic_44, singapore_genetic_runtime_44, singapore_actual_44)
print_results("Spain GP", spain_bayesian_44, spain_bayesian_runtime_44, spain_genetic_44, spain_genetic_runtime_44, spain_actual_44)

=== Mexico GP Results ===
Bayesian Optimization Runtime: 764.66 seconds
Top 10 Bayesian Optimization Strategies:
  1. Strategy: {1: np.int64(3), 35: np.int64(2)}, Final Position: 4.0
  2. Strategy: {1: np.int64(3), 34: np.int64(2), 47: np.int64(2)}, Final Position: 4.0
  3. Strategy: {1: np.int64(3), 34: np.int64(2)}, Final Position: 4.0
  4. Strategy: {1: np.int64(3), 34: np.int64(2), 47: np.int64(2)}, Final Position: 4.0
  5. Strategy: {1: np.int64(3), 37: np.int64(2), 46: np.int64(2)}, Final Position: 4.0
  6. Strategy: {1: np.int64(3), 37: np.int64(2), 47: np.int64(2)}, Final Position: 4.0
  7. Strategy: {1: np.int64(3), 38: np.int64(2), 48: np.int64(2)}, Final Position: 4.0
  8. Strategy: {1: np.int64(2), 9: np.int64(3), 55: np.int64(2)}, Final Position: 4.0
  9. Strategy: {1: np.int64(3), 37: np.int64(2), 47: np.int64(2)}, Final Position: 4.0
  10. Strategy: {1: np.int64(3), 37: np.int64(2), 48: np.int64(3)}, Final Position: 4.0

Genetic Algorithm Runtime: 786.11 seconds
Top 10 G

In [12]:
print_results("Mexico GP", mexico_bayesian_63, mexico_bayesian_runtime_63, mexico_genetic_63, mexico_genetic_runtime_63, mexico_actual_63)
print_results("Singapore GP", singapore_bayesian_63, singapore_bayesian_runtime_63, singapore_genetic_63, singapore_genetic_runtime_63, singapore_actual_63)
print_results("Spain GP", spain_bayesian_63, spain_bayesian_runtime_63, spain_genetic_63, spain_genetic_runtime_63, spain_actual_63)

=== Mexico GP Results ===
Bayesian Optimization Runtime: 769.44 seconds
Top 10 Bayesian Optimization Strategies:
  1. Strategy: {1: np.int64(2), 3: np.int64(2), 41: np.int64(3)}, Final Position: 4.0
  2. Strategy: {1: np.int64(3), 3: np.int64(2)}, Final Position: 5.0
  3. Strategy: {1: np.int64(2), 32: np.int64(2), 45: np.int64(3)}, Final Position: 5.0
  4. Strategy: {1: np.int64(3), 20: np.int64(2), 42: np.int64(3)}, Final Position: 5.0
  5. Strategy: {1: np.int64(2), 27: np.int64(3)}, Final Position: 5.0
  6. Strategy: {1: np.int64(3), 17: np.int64(2)}, Final Position: 5.0
  7. Strategy: {1: np.int64(2), 22: np.int64(2), 40: np.int64(3)}, Final Position: 5.0
  8. Strategy: {1: np.int64(3), 19: np.int64(2)}, Final Position: 5.0
  9. Strategy: {1: 1, 8: np.int64(2)}, Final Position: 5.0
  10. Strategy: {1: np.int64(3), 11: np.int64(2)}, Final Position: 5.0

Genetic Algorithm Runtime: 767.27 seconds
Top 10 Genetic Algorithm Strategies:
  1. Strategy: {1: np.int64(2), 54: np.int64(2), 21